# Initialisation

In [59]:
countries = [
    "Brazil",
    "Spain",
    "France",
    "Argentina",
    "Uruguay",
    "Colombia",
    "United Kingdom",
    "Paraguay",
    "Germany",
    "Ecuador"
]

elo_ratings = [
    1994,
    2150,
    2031,
    2140,
    1922,
    1953,
    2012,
    1799,
    1988,
    1911,
]

def get_elo(name):
    return countriesratings[name]

countriesratings = {country: elo_ratings[countries.index(country)] for country in countries}
countries_ranked = sorted(countries, key=get_elo, reverse=True)

locations = [
    (-14.2350, -51.9253),  # Brazil
    (40.4637, -3.7492),    # Spain
    (46.6034, 1.8883),     # France
    (-38.4161, -63.6167),  # Argentina
    (-32.5228, -55.7659),  # Uruguay
    (4.5709, -74.2973),    # Colombia
    (55.3781, -3.4360),    # United Kingdom
    (-23.4420, -58.4438),  # Paraguay
    (51.1657, 10.4515),    # Germany
    (-1.8312, -78.1834),   # Ecuador
]

locationdict = {country: locations[countries.index(country)] for country in countries}

# Distance calculation

In [60]:
from math import radians, sin, cos, sqrt, atan2

# DISTANCE CALCULATIONS
def haversine(pos1, pos2):
    lat1, lon1 = pos1
    lat2, lon2 = pos2
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    R = 6371.0 # Radius of Earth in kilometers
    return R * c # Distance in kilometers


def estimate_flight_time(pos1, pos2, speed_kmh=900):
    distance = haversine(pos1, pos2)
    time_hours = distance / speed_kmh
    return time_hours

# Schedule

In [61]:
from random import shuffle

def is_valid_schedule(schedule, countriesratings):
    home_away_count = {country: {"Home": 0, "Away": 0} for country in countries}
    
    for i in range(len(schedule)):
        country1, country2 = schedule[i]
        home_away_count[country1]["Home"] += 1
        home_away_count[country2]["Away"] += 1
        
        # Check for 3 consecutive home/away games
        if i >= 2:
            prev1, prev2 = schedule[i - 1]
            prev2_1, prev2_2 = schedule[i - 2]
            if (country1 == prev1 and country1 == prev2_1) or (country2 == prev2 and country2 == prev2_2):
                return False
        
        # Check for 3 consecutive games with rating difference > 250
        if i >= 2:
            diff1 = abs(countriesratings[schedule[i][0]] - countriesratings[schedule[i][1]])
            diff2 = abs(countriesratings[schedule[i - 1][0]] - countriesratings[schedule[i - 1][1]])
            diff3 = abs(countriesratings[schedule[i - 2][0]] - countriesratings[schedule[i - 2][1]])
            if diff1 > 250 and diff2 > 250 and diff3 > 250:
                return False
        
        # Ensure even number of home and away games
        for country, counts in home_away_count.items():
            if abs(counts["Home"] - counts["Away"]) > 1:
                return False
    
    return True

def generate_schedule(countries, countriesratings):
    all_matches = [(c1, c2) for i, c1 in enumerate(countries) for c2 in countries[i + 1:]]
    shuffle(all_matches)  # Randomize the order of matches

    valid_schedule = []

    for match in all_matches:
        country1, country2 = match

        # Ensure the schedule remains valid
        if is_valid_schedule(valid_schedule + [match], countriesratings):
            valid_schedule.append(match)

    return valid_schedule

# Generate the schedule
tournament_schedule = generate_schedule(countries, countriesratings)

# Display the matchups with home and away designation
print("Tournament Schedule:")
for match in tournament_schedule:
    home, away = match
    print(f"{home} (Home) vs {away} (Away)")


Tournament Schedule:
Spain (Home) vs Germany (Away)
Brazil (Home) vs Spain (Away)
Paraguay (Home) vs Ecuador (Away)
France (Home) vs Uruguay (Away)
Argentina (Home) vs Colombia (Away)
Colombia (Home) vs United Kingdom (Away)
Uruguay (Home) vs Colombia (Away)
United Kingdom (Home) vs Paraguay (Away)
Colombia (Home) vs Paraguay (Away)
Uruguay (Home) vs United Kingdom (Away)
Spain (Home) vs France (Away)
